# Decision Tree Classifier

In [1]:
# Import the libraries

import numpy as np
import pandas as pd

In [2]:
# A leaf of the tree that contains data points that all share the same target value is called pure. 
# Checks if the node is pure by returing true if the target column contains only one unqiue class

def check_purity(data):
    
    target_column = data[:, -1]
    unique_classes = np.unique(target_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [3]:
# Returns the target class which appears most often

def classify_data(data):
    
    target_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(target_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [4]:
# Returns the dictionary with all candidate splits for each respective column

def get_candidate_splits(data):
    
    candidate_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        candidate_splits[column_index] = unique_values
    
    return candidate_splits

In [5]:
# Split the dataset in order to create branches and it returns two datasets after splitting

def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_0 = data[split_column_values <= split_value]
        data_1 = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_0 = data[split_column_values == split_value]
        data_1 = data[split_column_values != split_value]
    
    return data_0, data_1

In [6]:
# Calculating entropy with weighted sums

def calculate_entropy(data):
    
    target_column = data[:, -1]
    _, counts = np.unique(target_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [7]:
# weighted sum of the entropies for the individual subsets

def calculate_overall_entropy(data_0, data_1):
    
    n = len(data_0) + len(data_1)
    p_data_0 = len(data_0) / n
    p_data_1 = len(data_1) / n

    overall_entropy =  (p_data_0 * calculate_entropy(data_0) 
                      + p_data_1 * calculate_entropy(data_1))
    
    return overall_entropy

In [8]:
#  We use entropy reduction to select the optimal split

def determine_best_split(data, candidate_splits):
    
    overall_entropy = 9999
    for column_index in candidate_splits:
        for value in candidate_splits[column_index]:
            data_0, data_1 = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_0, data_1)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [9]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "target":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

In [10]:
def decision_tree_algorithm(df, counter=0, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (counter == max_depth):
        classification = classify_data(data)
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        candidate_splits = get_candidate_splits(data)
        split_column, split_value = determine_best_split(data, candidate_splits)
        data_0, data_1 = split_data(data, split_column, split_value)
        
        # check for empty data
        if len(data_0) == 0 or len(data_1) == 0:
            classification = classify_data(data)
            return classification
        
        # determine decision
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            decision = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            decision = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {decision: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_0, counter, max_depth)
        no_answer = decision_tree_algorithm(data_1, counter, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[decision].append(yes_answer)
            sub_tree[decision].append(no_answer)
        
        return sub_tree

In [11]:
def predict(example, tree):
    decision = list(tree.keys())[0]
    feature_name, comparison_operator, value = decision.split(" ")

    # ask decision
    if comparison_operator == "<=":  # feature is continuous
        if example[feature_name] <= float(value):
            answer = tree[decision][0]
        else:
            answer = tree[decision][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[decision][0]
        else:
            answer = tree[decision][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return predict(example, residual_tree)

In [12]:
def calculate_accuracy(df, tree):

    df["classification"] = df.apply(predict, axis=1, args=(tree,))
    df["classification_correct"] = df["classification"] == df["target"]
    accuracy = df["classification_correct"].mean()
    
    return accuracy, df

In [13]:
import random
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [14]:
df = pd.read_csv("data/titanic.csv")
print("Data Size", df.shape)
df["target"] = df.Survived
df = df.drop(["PassengerId", "Survived", "Name", "Ticket", "Cabin", 'SibSp', 'Parch'], axis=1)

# handling missing values
median_age = df.Age.median()
mode_embarked = df.Embarked.mode()[0]

df = df.fillna({"Age": median_age, "Embarked": mode_embarked})

Data Size (891, 12)


In [15]:
from pprint import pprint
random.seed(0)
train_df, test_df = train_test_split(df, test_size=20)
tree = decision_tree_algorithm(train_df, max_depth=3)
accuracy, df = calculate_accuracy(test_df, tree)

pprint(tree)

{'Sex = male': [{'Fare <= 15.05': [{'Age <= 12.0': [1, 0]},
                                   {'Age <= 3.0': [1, 0]}]},
                {'Pclass = 3': [{'Fare <= 23.25': [1, 0]}, 1]}]}


In [16]:
print("Accuracy: ", accuracy * 100)
df.head(20)

Accuracy:  75.0


,Pclass,Sex,Age,Fare,Embarked,target,classification,classification_correct
864,2,male,24.0,13.0000,S,0,0,True
394,3,female,24.0,16.7000,S,1,1,True
776,3,male,28.0,7.7500,Q,0,0,True
430,1,male,28.0,26.5500,S,1,0,False
41,2,female,27.0,21.0000,S,0,1,False
265,2,male,36.0,10.5000,S,0,0,True
523,1,female,44.0,57.9792,C,1,1,True
497,3,male,28.0,15.1000,S,0,0,True
414,3,male,44.0,7.9250,S,1,0,False
802,1,male,11.0,120.0000,S,1,0,False


In [17]:
data = [['Medium', 'High', 75, 'Good'],
        ['Low', 'Low', 50, 'Bad'],
        ['High', 'Medium', 25, 'Bad'],
        ['Medium', 'Medium', 50, 'Good'],
        ['Low', 'Medium', 100, 'Good'],
        ['High', 'High', 25, 'Good'],
        ['Low', 'Low', 25, 'Bad'],
        ['Medium', 'Medium', 75, 'Good']]
 
# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Savings', 'Assets', 'Income', 'target'])

In [18]:
tree = decision_tree_algorithm(df, max_depth=3)
print(tree)

{'Assets = Low': ['Bad', {'Income = 25': [{'Assets = Medium': ['Bad', 'Good']}, 'Good']}]}


In [19]:
accuracy, df = calculate_accuracy(df, tree)
print("Accuracy: ", accuracy * 100)
df.head(10)

Accuracy:  100.0


,Savings,Assets,Income,target,classification,classification_correct
0,Medium,High,75,Good,Good,True
1,Low,Low,50,Bad,Bad,True
2,High,Medium,25,Bad,Bad,True
3,Medium,Medium,50,Good,Good,True
4,Low,Medium,100,Good,Good,True
5,High,High,25,Good,Good,True
6,Low,Low,25,Bad,Bad,True
7,Medium,Medium,75,Good,Good,True
